<img width="8%" alt="Plotly.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Plotly.png" style="border-radius: 15%">

# Plotly - Follow cash flow

**Tags:** #plotly #html #csv #image #finance #analytics #transactions #ledger #metric

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** This notebook creates a barline chart graph tracking your cashflows over the last 12 months.

## Input

### Import libraries

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import naas
import random
import os
from datetime import date, datetime, timedelta
import naas_data_product

### Setup variables
**Inputs**
- `entity_dir`: Entity directory.
- `entity_name`: Entity name.
- `input_dir`: Input directory to retrieve file from.
- `input_file`: Input file.
- `spreadsheet_url`: Google Sheets spreadsheet URL.
- `sheet_name`: Google Sheets sheet name.
- `title`: Graph title.

**Outputs**
- `output_dir`: This variable is used for storing the path to the directory where the output files will be saved.

In [ ]:
# Inputs
entity_dir = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", "0"), "entity_dir")
entity_name = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", "0"), "entity_name")
input_dir = os.path.join(entity_dir, "finance-engine", date.today().isoformat())
input_file = "transactions"
spreadsheet_url = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", "0"), "abi_spreadsheet")
sheet_name = "TRANSACTIONS"
title = "Cash Flow Statement"

# Outputs
output_dir = os.path.join(entity_dir, "finance-engine", date.today().isoformat())
os.makedirs(output_dir, exist_ok=True)
output_name = "finance_trend"

## Model

### Set outputs

In [ ]:
html_output = os.path.join(output_dir, f"{output_name}.html")
image_output = os.path.join(output_dir, f"{output_name}.png")

### Get "Cashin" data

In [ ]:
data_cashin = {
    "ENTITY": ["Abi"] * 12,
    "SCENARIO": ["2023-12"] * 12,
    "LABEL": [
        "2023-01",
        "2023-02",
        "2023-03",
        "2023-04",
        "2023-05",
        "2023-06",
        "2023-07",
        "2023-08",
        "2023-09",
        "2023-10",
        "2023-11",
        "2023-12",
    ],
    "GROUP": ["Cash in"] * 12,
    "VALUE": [random.randint(250, 500) for i in range(0, 12)],
}

df_cashin = pd.DataFrame(data_cashin)
df_cashin

### Get "Cashout" data

In [ ]:
data_cashout = {
    "ENTITY": ["Abi"] * 12,
    "SCENARIO": ["2023-12"] * 12,
    "LABEL": [
        "2023-01",
        "2023-02",
        "2023-03",
        "2023-04",
        "2023-05",
        "2023-06",
        "2023-07",
        "2023-08",
        "2023-09",
        "2023-10",
        "2023-11",
        "2023-12",
    ],
    "GROUP": ["Cash out"] * 12,
    "VALUE": [random.randint(0, 350) * -1 for i in range(0, 12)],
}
df_cashout = pd.DataFrame(data_cashout)
df_cashout

### Calculate "Cash Position"

In [ ]:
# concat cash in and cash out
df_position = pd.concat([df_cashin, df_cashout])

# rename column GROUP = "Position" and groupby + agg
to_group = [
    "ENTITY",
    "SCENARIO",
    "LABEL",
    "GROUP"
]
to_agg = {
    "VALUE": "sum"
}
df_position["GROUP"] = "Position"
df_position["VALUE"] = df_position["VALUE"] + 1000
df_position = df_position.groupby(to_group, as_index=False).agg(to_agg)
df_position

### Create title and logo

In [ ]:
# Calc data
total = df_position.loc[df_position.index[-1], "VALUE"]
total_n1 = df_position.loc[df_position.index[-2], "VALUE"]
varv = df_cashin.loc[df_cashin.index[-1], "VALUE"] + df_cashout.loc[df_cashout.index[-1], "VALUE"]
varp = 0
if total_n1 != 0:
    varp = varv / total_n1

# Create value to displayed
total_d = "{:,.0f}".format(total).replace(",", " ")
varv_d = "{:,.0f}".format(varv).replace(",", " ")
varp_d = "{:,.0%}".format(varp).replace(",", " ")
if varv >= 0:
    varv_d = f"+{varv_d}"
    varp_d = f"+{varp_d}"
title_full = f"<b><span style='font-size: 20px;'>{title}</span></b><br><span style='font-size: 18px;'>{total_d} | {varv_d} ({varp_d}) vs last month</span>"

# Logo
logo = None
if varv > 0:
    logo = arrow_up
elif varv > -0.2:
    logo = arrow_right
else:
    logo = arrow_down
print("Title:", title_full)

### Create barlinechart using Plotly

In [ ]:
def create_barlinechart(
    df_cashin,
    df_cashout,
    df_position,
    xaxis_title=None,
    yaxis_title_r=None,
    yaxis_title_l=None,
):
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Bar(
            name="Encaissement",
            x=df_cashin["LABEL"],
            y=df_cashin["VALUE"],
            marker=dict(color="#1b7656"),
        ),
        secondary_y=False,
    )
    fig.add_trace(
        go.Bar(
            name="Décaissement",
            x=df_cashout["LABEL"],
            y=df_cashout["VALUE"] * -1,
            marker=dict(color="#cd3244"),
        ),
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(
            x=df_position["LABEL"],
            y=df_position["VALUE"],
            mode="lines",
            line=dict(color="#46a7f5", width=2.5),
        ),
        secondary_y=True,
    )
    # Add logo
    fig.add_layout_image(
        dict(
            source=logo,
            xref="paper",
            yref="paper",
            x=0.01,
            y=1.05,
            sizex=0.12,
            sizey=0.12,
            xanchor="right",
            yanchor="bottom",
        )
    )

    # Add figure title
    fig.update_layout(
        title=title_full,
        title_x=0.09,
        title_font=dict(family="Arial", color="black"),
        paper_bgcolor="#ffffff",
        plot_bgcolor="#ffffff",
        legend=None,
        margin_pad=10,
        margin_r=10,
        width=1200,
        height=600,
        xaxis_title=xaxis_title,
        xaxis_title_font=dict(family="Arial", size=12, color="black"),
        xaxis={"type": "category"},
    )

    # Set y-axes titles
    fig.update_yaxes(
        title_text=yaxis_title_r,
        title_font=dict(family="Arial", size=12, color="black"),
        secondary_y=False,
    )
    fig.update_yaxes(
        title_text=yaxis_title_l,
        title_font=dict(family="Arial", size=12, color="black"),
        secondary_y=True,
    )
    fig.update_traces(showlegend=False)
    fig.show()
    return fig


fig = create_barlinechart(
    df_cashin,
    df_cashout,
    df_position,
    xaxis_title=None,
    yaxis_title_r="Flows",
    yaxis_title_l="Position",
)

## Output

### Save and share your csv file

In [ ]:
# # Save your dataframe in CSV
# df_trend.to_csv(csv_output, index=False)

### Save and share your graph in HTML


In [ ]:
# Save your graph in HTML
fig.write_html(html_output)

# Share output with naas
html_link = naas.asset.add(html_output, override_prod=True, params={"inline": True})

# -> Uncomment the line below to remove your asset
# naas.asset.delete(html_output)

### Save and share your graph in image


In [ ]:
# Save your graph in PNG
fig.write_image(image_output)

# Share output with naas
image_link = naas.asset.add(image_output, override_prod=True, params={"inline": True})

# -> Uncomment the line below to remove your asset
# naas.asset.delete(image_output)